In [1]:
!pip install transformers datasets evaluate accelerate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 9.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 12.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 297.3/297.3 kB 30.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 16.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 25.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 17.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 43.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 66.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 85.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 1.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 3.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 8.3 MB/s eta 0:00:00
     ━━━━━━━━━

In [2]:
import pandas as pd

df = pd.read_csv('spam.csv')
df.head(5)

,Category,Message
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [3]:
df.groupby('Category').describe()

Message                                                            \
           count unique                                                top   
Category                                                                     
ham         4825   4516                             Sorry, I'll call later   
spam         747    641  Please call our customer service representativ...   

               
         freq  
Category       
ham        30  
spam        4

In [4]:
df_spam = df[df['Category']=='spam']
df_spam.shape

(747, 2)

In [5]:
df_ham = df[df['Category']=='ham']
df_ham.shape

(4825, 2)

In [6]:
df_ham_downsampled = df_ham.sample(df_spam.shape[0])
df_ham_downsampled.shape

(747, 2)

In [7]:
df_balanced = pd.concat([df_ham_downsampled, df_spam])
df_balanced.shape

(1494, 2)

In [8]:
df_balanced['Category'].value_counts()

Category
ham     747
spam    747
Name: count, dtype: int64

In [9]:
# Convert the category to a binary variable
df_balanced['spam'] = df_balanced['Category'].apply(lambda x: 1 if x=='spam' else 0)

In [10]:
# drop the old category column
df_balanced.drop('Category', axis='columns', inplace=True)
df_balanced.head()

,Message,spam
2584,"Goodmorning, today i am late for 1hr.",0
4539,Dare i ask... Any luck with sorting out the car?,0
1744,I love to wine and dine my lady!,0
1477,I'm watching lotr w my sis dis aft. So u wan 2...,0
1068,Meeting u is my work. . . Tel me when shall i ...,0


In [11]:
# rename columns to 'text' and 'label'
df_balanced.rename(columns={'Message': 'text'}, inplace=True)
df_balanced.rename(columns={'spam': 'label'}, inplace=True)
df_balanced.head()

,text,label
2584,"Goodmorning, today i am late for 1hr.",0
4539,Dare i ask... Any luck with sorting out the car?,0
1744,I love to wine and dine my lady!,0
1477,I'm watching lotr w my sis dis aft. So u wan 2...,0
1068,Meeting u is my work. . . Tel me when shall i ...,0


In [12]:
# Split the data into training and testing
from sklearn.model_selection import train_test_split
train_data, test_data = train_test_split(df_balanced, test_size=0.2, random_state=42)

In [13]:
print(train_data.shape)
print(test_data.shape)

(1195, 2)
(299, 2)


In [32]:
from datasets import Dataset
ds_train = Dataset.from_dict({"text": train_data['text'], "label": train_data['label']})
ds_test = Dataset.from_dict({"text": test_data['text'], "label": test_data['label']})


In [33]:
print(ds_test)

Dataset({
    features: ['text', 'label'],
    num_rows: 299
})


In [34]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("distilbert/distilbert-base-uncased")

In [35]:
def preprocess_function(examples):
    return tokenizer(examples["text"], truncation=True)

In [37]:
tokenized_data_train = ds_train.map(preprocess_function, batched=True)
tokenized_data_test = ds_test.map(preprocess_function, batched=True)

Map:   0%|          | 0/1195 [00:00<?, ? examples/s]

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


Map:   0%|          | 0/299 [00:00<?, ? examples/s]

In [38]:
from transformers import DataCollatorWithPadding

data_collator = DataCollatorWithPadding(tokenizer=tokenizer, return_tensors="tf")

In [39]:
import evaluate

accuracy = evaluate.load("accuracy")

In [40]:
import numpy as np


def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return accuracy.compute(predictions=predictions, references=labels)

In [41]:
id2label = {0: "ham", 1: "spam"}
label2id = {"ham": 0, "spam": 1}

In [42]:
from transformers import create_optimizer
import tensorflow as tf

batch_size = 16
num_epochs = 5
batches_per_epoch = len(tokenized_data_train) // batch_size
total_train_steps = int(batches_per_epoch * num_epochs)
optimizer, schedule = create_optimizer(init_lr=2e-5, num_warmup_steps=0, num_train_steps=total_train_steps)

In [43]:
# Check TensorFlow and Transformers versions
import tensorflow as tf
import transformers

print("TensorFlow version:", tf.__version__)
print("Transformers version:", transformers.__version__)

TensorFlow version: 2.15.0
Transformers version: 4.38.2


In [44]:
from transformers import TFAutoModelForSequenceClassification

model = TFAutoModelForSequenceClassification.from_pretrained(
    "distilbert/distilbert-base-uncased", num_labels=2, id2label=id2label, label2id=label2id
)

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFDistilBertForSequenceClassification: ['vocab_projector.bias', 'vocab_layer_norm.bias', 'vocab_transform.bias', 'vocab_transform.weight', 'vocab_layer_norm.weight']
- This IS expected if you are initializing TFDistilBertForSequenceClassification from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertForSequenceClassification from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
Some weights or buffers of the TF 2.0 model TFDistilBertForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['pre_classifier.weight', 'pre_classifier.bias', 'classifier.weight', 'classifier.bias']
You should 

In [45]:
tf_train_set = model.prepare_tf_dataset(
    tokenized_data_train,
    shuffle=True,
    batch_size=16,
    collate_fn=data_collator,
)

tf_validation_set = model.prepare_tf_dataset(
    tokenized_data_test,
    shuffle=False,
    batch_size=16,
    collate_fn=data_collator,
)

In [46]:
import tensorflow as tf

model.compile(optimizer=optimizer)  # No loss argument!

In [47]:
from transformers.keras_callbacks import KerasMetricCallback

metric_callback = KerasMetricCallback(metric_fn=compute_metrics, eval_dataset=tf_validation_set)

In [48]:
callbacks = [metric_callback]

In [49]:
model.fit(x=tf_train_set, validation_data=tf_validation_set, epochs=3, callbacks=callbacks)

Epoch 1/3


Cause: for/else statement not yet supported
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


Cause: for/else statement not yet supported
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
74/74 [==============================] - 92s 579ms/step - loss: 0.2415 - val_loss: 0.0792 - accuracy: 0.9833
Epoch 2/3
74/74 [==============================] - 13s 178ms/step - loss: 0.0814 - val_loss: 0.0696 - accuracy: 0.9799
Epoch 3/3
74/74 [==============================] - 12s 163ms/step - loss: 0.0383 - val_loss: 0.0576 - accuracy: 0.9833


In [50]:
model.save('spam-detector')

In [52]:
!pip install tensorflowjs

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.1/89.1 kB 1.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.5/15.5 MB 18.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.0/53.0 kB 7.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 589.8/589.8 MB 2.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 96.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.8/4.8 MB 94.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 67.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 103.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 81.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 311.2/311.2 kB 40.0 MB/s eta 0:00:00
  Attempting uninstall: packaging
    Found existing installation: packaging 24.0
    Uninstalling packaging-24.0:
      Successfully uninstalled packaging-24.0
  Attempting uninstall: ml-dtypes
   

In [53]:
!tensorflowjs_converter --input_format=tf_saved_model \
                       --output_format=tfjs_layers_model \
                       spam-detector \
                       spam-detector-js


2024-04-07 21:19:37.594362: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
Traceback (most recent call last):
  File "/usr/local/bin/tensorflowjs_converter", line 8, in <module>
    sys.exit(pip_main())
  File "/usr/local/lib/python3.10/dist-packages/tensorflowjs/converters/converter.py", line 958, in pip_main
    main([' '.join(sys.argv[1:])])
  File "/usr/local/lib/python3.10/dist-packages/tensorflowjs/converters/converter.py", line 962, in main
    convert(argv[0].split(' '))
  File "/usr/local/lib/python3.10/dist-packages/tensorflowjs/converters/converter.py", line 948, in convert
    _dispatch_converter(input_format, output_format, args, quantization_dtype_map,
  File "/usr/local/lib/python3.10/dist-packages/tensorflowjs/converters/converter.py", line 720, in _dispatch_converter
    raise ValueError(
ValueError: Unsupported input_format - output_format pair: tf_saved_model - tfjs_layers_model


In [57]:
!zip -r /content/drive/MyDrive/model.zip spam-detector/

  adding: spam-detector/ (stored 0%)
  adding: spam-detector/variables/ (stored 0%)
  adding: spam-detector/variables/variables.data-00000-of-00001 (deflated 27%)
  adding: spam-detector/variables/variables.index (deflated 78%)
  adding: spam-detector/fingerprint.pb (stored 0%)
  adding: spam-detector/saved_model.pb (deflated 92%)
  adding: spam-detector/keras_metadata.pb (deflated 94%)
  adding: spam-detector/assets/ (stored 0%)


In [56]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
